In [1]:
# test_imports in this cell, restart kernel whenever edits to modules are made

import cooltools
import cooler
import sys
import os
from matplotlib.colors import LinearSegmentedColormap
from matplotlib import pyplot as plt
from matplotlib import gridspec
import numpy as np
import seaborn as sns



# Append the src directory to the path
current_dir = os.getcwd()
src_dir = os.path.abspath(os.path.join(current_dir, '..', 'src'))
if src_dir not in sys.path:
    sys.path.append(src_dir)

#import config based on which machine the jupyter notebook is running on
from configs.config_local import *
from preprocessing import *
from module_detection import *
from utils import *

def test_imports():
    print("testing config class")
    config = Config()
    print(config)

    print("testing 2d contacts preprocessing classes")
    query = HiCQuery(config, 'chr1', 1000000, '1Mb') #needs config, chrom and res/res0 for instantiation
    print(query)

    
    #other prints for imported classes

test_imports()

testing config class
testing 2d contacts preprocessing classes
HiC file loaded


In [2]:
## check how much a/b from cooltools bigwig aligns with a/b from my code visually ##

#initialising parameters for single chromosome analysis

config = Config()
chrom = config.genomic_params.chromosomes[0]
current_res = config.genomic_params.resolutions[0]  # 1Mb for OE part
current_res_str = config.genomic_params.res_strs[0]  # 1Mb for OE part
query = HiCQuery(config, chrom, current_res, current_res_str)

HiC file loaded


In [ ]:
def plot_2dhic_1dtrack_map(dense_matrix, signal, vmin, vmax, cmap):
  d2 = dense_matrix # np.log(dense_matrix)
  d2[np.isnan(d2)] = 0
  d2[np.isinf(d2)] = 0
  fig = plt.figure()
  fig.set_figheight(12)
  fig.set_figwidth(8)
  spec = gridspec.GridSpec(ncols=1, nrows=2,
                         width_ratios=[1], wspace=1,
                         hspace=.1, height_ratios=[16, 30])
  ax0 = fig.add_subplot(spec[0])
  ax0.plot(np.arange(len(signal)), signal)
  ax1 = fig.add_subplot(spec[1])
  ax1.matshow(d2, cmap=cmap, vmin=vmin, vmax=vmax)
  plt.show()

# params for OE matrix visualisation
threshold = 0
start = 0
end = 72000000

# Custom colormap
REDMAP = LinearSegmentedColormap.from_list("bright_red", [(1, 1, 1), (1, 0, 0)])

## single chr visualisation ##
oe_numpy_thresholded = query.oe_intra_numpy(start, end, threshold)

# params for AB score visualisation along with OE matrix
ab_geo_bw = pyBigWig.open(config.paths.compartments_infile)
nm_bins = oe_numpy_thresholded.shape[0]
signal = ab_geo_bw.stats(
                chrom, start, end, type="mean", nBins=nm_bins
            )
#filter none values
filtered_signal = [0 if x is None else x for x in signal]
plot_2dhic_1dtrack_map(oe_numpy_thresholded, filtered_signal, 0, 1, REDMAP)